In [ ]:
import sys
sys.path.append('../') # or just install the module
sys.path.append('../../fuzzy-tools') # or just install the module
sys.path.append('../../astro-lightcurves-handler') # or just install the module

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from fuzzytools.files import load_pickle, save_pickle, get_dict_from_filedir

def get_lcdataset(method,
    verbose=1,
    ):
    filedir = f'../../surveys-save/survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method={method}.splcds'
    if verbose:
        print(filedir)
    filedict = get_dict_from_filedir(filedir)
    root_folder = filedict['_rootdir']
    cfilename = filedict['_cfilename']
    survey = filedict['survey']
    lcdataset = load_pickle(filedir)
    return lcdataset


methods = [
    'linear-fstw',
    'bspline-fstw',
    'spm-mcmc-estw',
    'spm-mcmc-fstw',
    ]
kfs = list(range(0, 5))
for method in methods:
    lcdataset = get_lcdataset(method)
    for kf in kfs:
        print(f'kf={kf}')
        set_names = [f'{kf}@train', f'{kf}@val', f'{kf}@test', f'{kf}@train.{method}']
        for set_name in set_names:
            lcobjs = lcdataset[set_name].get_lcobjs()
            total_obs = 0
            for lcobj in lcobjs:
                band_names = lcobj.bands
                days = []
                for b in band_names:
                    days += lcobj.get_b(b).days.tolist()
                min_day = min(days)
                assert min_day==0
                total_obs += len(lcobj)
            print(f'\tset_name={set_name}; total_obs={total_obs}')

In [ ]:
%load_ext autoreload
%autoreload 2
import numpy as np
from fuzzytools.files import load_pickle, save_pickle, get_dict_from_filedir
from fuzzytools.lists import check_lists_are_different, check_lists_are_equal, get_nof_common_elements

def get_lcdataset(method,
    verbose=1,
    ):
    filedir = f'../../surveys-save/survey=alerceZTFv7.1~bands=gr~mode=onlySNe~method={method}.splcds'
    if verbose:
        print(filedir)
    filedict = get_dict_from_filedir(filedir)
    root_folder = filedict['_rootdir']
    cfilename = filedict['_cfilename']
    survey = filedict['survey']
    lcdataset = load_pickle(filedir)
    return lcdataset


methods = [
    'linear-fstw',
    'bspline-fstw',
    'spm-mcmc-estw',
    'spm-mcmc-fstw',
]
kfs = list(range(0, 5))
for method in methods:
    lcdataset = get_lcdataset(method)
    for kf in kfs:
        print(f'kf={kf}')
        train_lcobj_names = lcdataset[f'{kf}@train'].get_lcobj_names()
        synth_train_lcobj_names = lcdataset[f'{kf}@train.{method}'].get_lcobj_names()
        clean_synth_train_lcobj_names = list(set([n.split('.')[0] for n in synth_train_lcobj_names]))
        val_lcobj_names = lcdataset[f'{kf}@val'].get_lcobj_names()
        test_lcobj_names = lcdataset[f'{kf}@test'].get_lcobj_names()

        print(f'\ttrain {len(train_lcobj_names)}/train {len(train_lcobj_names)}')
        assert check_lists_are_equal(train_lcobj_names, train_lcobj_names)  # sanity check
        assert get_nof_common_elements(train_lcobj_names, train_lcobj_names) == len(train_lcobj_names)

        print(f'\ttrain {len(train_lcobj_names)}/val {len(val_lcobj_names)}')
        assert check_lists_are_different(train_lcobj_names, val_lcobj_names)
        assert get_nof_common_elements(train_lcobj_names, val_lcobj_names) == 0

        print(f'\ttrain {len(train_lcobj_names)}/test {len(test_lcobj_names)}')
        assert check_lists_are_different(train_lcobj_names, test_lcobj_names)
        assert get_nof_common_elements(train_lcobj_names, test_lcobj_names) == 0

        print(f'\tval {len(val_lcobj_names)}/test {len(test_lcobj_names)}')
        assert check_lists_are_different(val_lcobj_names, test_lcobj_names)
        assert get_nof_common_elements(val_lcobj_names, test_lcobj_names) == 0

        print(f'\ttrain {len(train_lcobj_names)}/synth_train {len(synth_train_lcobj_names)}')
        assert check_lists_are_different(train_lcobj_names, synth_train_lcobj_names)
        assert get_nof_common_elements(train_lcobj_names, synth_train_lcobj_names) == 0

        print(f'\ttrain {len(train_lcobj_names)}/clean_synth_train {len(clean_synth_train_lcobj_names)}')
        assert check_lists_are_equal(train_lcobj_names, clean_synth_train_lcobj_names)
        assert get_nof_common_elements(train_lcobj_names, clean_synth_train_lcobj_names) == len(train_lcobj_names)

        print(f'\tclean_synth_train {len(clean_synth_train_lcobj_names)}/val {len(val_lcobj_names)}')
        assert check_lists_are_different(clean_synth_train_lcobj_names, val_lcobj_names)
        assert get_nof_common_elements(clean_synth_train_lcobj_names, val_lcobj_names) == 0

        print(f'\tclean_synth_train {len(clean_synth_train_lcobj_names)}/test {len(test_lcobj_names)}')
        assert check_lists_are_different(clean_synth_train_lcobj_names, test_lcobj_names)
        assert get_nof_common_elements(clean_synth_train_lcobj_names, test_lcobj_names) == 0

        for other_method in methods:
            if method==other_method:
                continue
            other_lcdataset = get_lcdataset(other_method, verbose=0)
            other_train_lcobj_names = other_lcdataset[f'{kf}@train'].get_lcobj_names()
            other_synth_train_lcobj_names = other_lcdataset[f'{kf}@train.{other_method}'].get_lcobj_names()
            other_val_lcobj_names = other_lcdataset[f'{kf}@val'].get_lcobj_names()
            other_test_lcobj_names = other_lcdataset[f'{kf}@test'].get_lcobj_names()

            print(f'\t\t(v/s {other_method}) train {len(train_lcobj_names)}/other_train {len(other_train_lcobj_names)}')
            assert check_lists_are_equal(train_lcobj_names, other_train_lcobj_names)
            assert get_nof_common_elements(train_lcobj_names, other_train_lcobj_names) == len(train_lcobj_names)

            print(f'\t\t(v/s {other_method}) synth_train {len(synth_train_lcobj_names)}/other_synth_train {len(other_synth_train_lcobj_names)}')
            assert check_lists_are_equal(synth_train_lcobj_names, other_synth_train_lcobj_names)
            assert get_nof_common_elements(synth_train_lcobj_names, other_synth_train_lcobj_names) == len(synth_train_lcobj_names)

            print(f'\t\t(v/s {other_method}) val {len(val_lcobj_names)}/other_val {len(other_val_lcobj_names)}')
            assert check_lists_are_equal(val_lcobj_names, other_val_lcobj_names)
            assert get_nof_common_elements(val_lcobj_names, other_val_lcobj_names) == len(val_lcobj_names)

            print(f'\t\t(v/s {other_method}) test {len(test_lcobj_names)}/other_test {len(other_test_lcobj_names)}')
            assert check_lists_are_equal(test_lcobj_names, other_test_lcobj_names)
            assert get_nof_common_elements(test_lcobj_names, other_test_lcobj_names) == len(test_lcobj_names)
    print()
print('done')